In [1]:
!pip install -q streamlit transformers torch pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

In [3]:
!pip install python-docx chardet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 12.4 MB/s eta 0:00:00


In [4]:
!pip install plotly


In [5]:
!pip install spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 66.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [9]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pwd

/content


In [47]:
%%writefile app.py
import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
import pandas as pd
import docx
import spacy
import plotly.graph_objects as go
import sys

print("🚀 Streamlit Python:", sys.executable, flush=True)

# ---- Page setup ----
st.set_page_config(page_title="Emotion Classifier", page_icon="🎯", layout="wide")

# ---- Header UI ----
st.markdown("""
    <h1 style='text-align: center; color: #FF8C00;'>🎯 Emotion Classifier</h1>
    <p style='text-align: center;'>Analyze emotional tone from text or document using DeBERTa</p>
    <hr style='border-top: 1px solid #bbb;'>
""", unsafe_allow_html=True)

# ---- Load spaCy ----
nlp = spacy.load("en_core_web_sm")

# ---- Model config ----
MODEL_PATH = "microsoft/deberta-v3-base"
LABELS = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral']
EMOTION_DESCRIPTIONS = {
    "joy": "Joy, happiness, contentment",
    "sadness": "Sadness, pain, grief",
    "anger": "Anger, frustration, rage",
    "surprise": "Surprise, shock, amazement",
    "disgust": "Disgust, repulsion, disapproval",
    "fear": "Fear, anxiety, worry",
    "neutral": "No clear emotion or neutral tone"
}

@st.cache_resource
def load_model():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH, num_labels=len(LABELS))
    state_dict = torch.load("/content/drive/MyDrive/deberta_goemotions_epoch1.pth", map_location=device)
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval()
    return tokenizer, model, device

def split_into_sentences(text):
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents if sent.text.strip()]

def plot_emotion_radar_plotly(labels, probs, title="Emotion Radar"):
    values = list(probs)
    dominant_label = labels[np.argmax(probs)]
    COLOR_BY_EMOTION = {
        "joy": "#FFD700", "sadness": "#4169E1", "anger": "#DC143C", "fear": "#800080",
        "surprise": "#00CED1", "disgust": "#556B2F", "neutral": "#AAAAAA"
    }
    line_color = COLOR_BY_EMOTION.get(dominant_label, "#00BFFF")

    fig = go.Figure()
    fig.add_trace(go.Scatterpolar(
        r=values + [values[0]],
        theta=labels + [labels[0]],
        fill='toself',
        name='Emotions',
        line=dict(color=line_color, width=3),
        marker=dict(size=8),
        hoverinfo='all'
    ))

    fig.update_layout(
        title=title,
        polar=dict(
            bgcolor="#1e1e1e",
            gridshape="linear",
            radialaxis=dict(visible=True, range=[0, 1], tickfont=dict(color='white')),
            angularaxis=dict(tickfont=dict(color='white'))
        ),
        showlegend=False,
        paper_bgcolor="#121212",
        font=dict(color="white")
    )
    return fig

def highlight_max_emotion(row):
    emotions = row[LABELS]
    max_idx = np.argmax(emotions)
    return [
        "background-color: rgba(249,136,102,0.2); font-weight: 500" if i == max_idx else ""
        for i in range(len(emotions))
    ]

# ---- Load model ----
tokenizer, model, device = load_model()

# ---- Tabs UI ----
tab1, tab2 = st.tabs(["📝 Text Input", "📁 Upload File"])

# ==== TAB 1: TEXT INPUT ====
with tab1:
    text = st.text_area("", placeholder="Enter your text here...")
    threshold = st.slider("Prediction threshold", 0.1, 0.9, 0.3, 0.05)

    if st.button("Predict", key="btn_text") and text.strip():
        sentences = split_into_sentences(text)
        st.markdown(f"🔍 Detected **{len(sentences)}** sentences.")

        results = []
        for sent in sentences:
            inputs = tokenizer(sent, return_tensors="pt", truncation=True, padding=True).to(device)
            with torch.no_grad():
                logits = model(**inputs).logits
                probs = torch.sigmoid(logits).squeeze().cpu().numpy()
            results.append([sent] + list(probs))

        df = pd.DataFrame(results, columns=["Sentence"] + LABELS)
        mean_probs = df[LABELS].mean().tolist()

        col1, col2 = st.columns([1, 2])

        with col1:
            st.subheader("🎯 Overall Emotion Summary")
            significant = [(l, p) for l, p in zip(LABELS, mean_probs) if p >= threshold]
            if significant:
                for label, prob in significant:
                    desc = EMOTION_DESCRIPTIONS.get(label, "")
                    st.markdown(f"- **{label.capitalize()}**: `{prob:.2f}` – _{desc}_")
            else:
                max_idx = int(np.argmax(mean_probs))
                label = LABELS[max_idx]
                st.markdown(f"- **{label.capitalize()}**: `{mean_probs[max_idx]:.2f}` (highest) – _{EMOTION_DESCRIPTIONS[label]}_")

        with col2:
            st.subheader("📊 Overall Emotion Radar")
            radar_buf = plot_emotion_radar_plotly(LABELS, mean_probs)
            st.plotly_chart(radar_buf, use_container_width=True, key="radar_text_input")

        st.subheader("📋 Sentence-level Predictions")
        styled_df = df.style.apply(highlight_max_emotion, axis=1, subset=LABELS)
        st.dataframe(styled_df, use_container_width=True)


# ==== TAB 2: FILE UPLOAD ====
with tab2:
    uploaded_file = st.file_uploader("Upload a .txt or .docx file", type=["txt", "docx"])
    threshold = st.slider("Prediction threshold", 0.1, 0.9, 0.3, 0.05, key="slider_file")

    if uploaded_file:
        if uploaded_file.name.endswith(".txt"):
            text = uploaded_file.read().decode("utf-8")
        elif uploaded_file.name.endswith(".docx"):
            doc = docx.Document(uploaded_file)
            text = "\n".join([para.text for para in doc.paragraphs])
        else:
            st.warning("Unsupported file format.")
            text = ""

        if text:
            sentences = split_into_sentences(text)
            st.markdown(f"🔍 Detected **{len(sentences)}** sentences.")

            results = []
            for sent in sentences:
                inputs = tokenizer(sent, return_tensors="pt", truncation=True, padding=True).to(device)
                with torch.no_grad():
                    logits = model(**inputs).logits
                    probs = torch.sigmoid(logits).squeeze().cpu().numpy()
                results.append([sent] + list(probs))

            df = pd.DataFrame(results, columns=["Sentence"] + LABELS)

            col1, col2 = st.columns([1, 2])

            with col1:
                st.subheader("🎯 Overall Emotion Summary")
                mean_probs = df[LABELS].mean().tolist()
                significant = [(l, p) for l, p in zip(LABELS, mean_probs) if p >= threshold]
                if significant:
                    for label, prob in significant:
                        desc = EMOTION_DESCRIPTIONS.get(label, "")
                        st.markdown(f"- **{label.capitalize()}**: `{prob:.2f}` – _{desc}_")
                else:
                    max_idx = int(np.argmax(mean_probs))
                    label = LABELS[max_idx]
                    st.markdown(f"- **{label.capitalize()}**: `{mean_probs[max_idx]:.2f}` (highest) – _{EMOTION_DESCRIPTIONS[label]}_")

            with col2:
                st.subheader("📊 Overall Emotion Radar")
                fig = plot_emotion_radar_plotly(LABELS, mean_probs)
                st.plotly_chart(fig, use_container_width=True, key="radar_file_input")

            st.subheader("📄 Sentence-level Predictions")
            styled_df = df.style.apply(highlight_max_emotion, axis=1, subset=LABELS)
            st.dataframe(styled_df, use_container_width=True)
            csv = df.to_csv(index=False).encode("utf-8")
            st.download_button("⬇️ Download Results (.csv)", data=csv, file_name="emotion_analysis.csv", mime="text/csv")


Overwriting app.py


In [11]:
!ngrok config add-authtoken 2zGqw93wHrCFlYVk8E0yiqgeQUe_6AeYt8RWweFj9Q237XaLE


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [49]:
from pyngrok import ngrok

!pkill -f streamlit
ngrok.kill()


In [50]:
import threading
import time

def run():
    !streamlit run app.py --server.headless true --server.port 8501

threading.Thread(target=run, daemon=True).start()

time.sleep(5)

public_url = ngrok.connect("http://localhost:8501")
print("🌐 App is live at:", public_url)




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.91.83.166:8501

🌐 App is live at: NgrokTunnel: "https://fb43-34-91-83-166.ngrok-free.app" -> "http://localhost:8501"
